# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This problem is a classification problem. The reason is that the goal is to divide students to 2 groups (early intervention or not) - 2 classes. We already know what the output classes are, we just need to find out that into which group the input should go. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [47]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


In [48]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = (student_data["passed"]=="yes").value_counts()[0]

# TODO: Calculate failing students
n_failed = (student_data["passed"]=="yes").value_counts()[1]

# TODO: Calculate graduation rate
grad_rate = (float(n_passed) / float(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 130
Number of students who failed: 265
Graduation rate of the class: 32.91%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [49]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [79]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

print "\nFeatures count: ", len(X_all.columns) 

print X_all.head(10)

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Features count:  48
   school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  famsize_GT3  \
0        1.0        0.0    1.0    0.0   18        0.0        1.0          1.0   
1        1.0        0.0    1.0    0.0   17        0.0        1.0          1.0   
2        1.0        0.0    1.0    0.0   15        0.0       

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [74]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

np.random.seed(10)
X_rand = X_all.reindex(np.random.permutation(X_all.index))

np.random.seed(10)
y_rand = y_all.reindex(np.random.permutation(y_all.index))


# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test = train_test_split(X_rand, test_size = num_test)
y_train, y_test = train_test_split(y_rand, test_size = num_test)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

Data: 
- Training size: 300
- Testing size: 95
- Features: 48



**Model A: Decision tree**
I think that the features should be a good candidates for a decision tree model. Reason is that I think that the features are linear and properly grouped (clear margin separation). For example everybody with grade avarage 4 above will pass (problem would be if everybody above grade avarege 6 would not pass)
- Pros
 - Data classification without much calculations
 - Dealing with noisy or incomplete data
- Cons
 - Exponential calculation growth while problem is getting bigger


**Model B: Support Vector Machines (SVM)**
I think that this model should be doing quite well because of the data should have clear margin separation + the data set is not very big. I'm expecting a quite good results from the SVM model.
- Pros
 - It works really well with clear margin of separation
 - It is effective in high dimensional spaces.
- Cons
 - It doesn’t perform well, when we have large data set because the required training time is higher
 - It also doesn’t perform very well, when the data set has more noise i.e. target classes are overlapping



**Model C: K-NN (Nearest Neigbour)**
I would like to try this model because of its simplicity and to find out how changes of k will influence the model results

- Pros
 - Simple & Effective
 - Fast training phase
- Cons
 - Does not produce a model, limiting the ability to understand how the features are related to the class
 - Requires selectioin of an appropriate k
 


**Model D: Logistic regression**
Just out of my interest I would like to see, that how well will do logisitic regression compared to other models.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [75]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [76]:
import numpy as np
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn import tree
# from sklearn import model_B
from sklearn import svm
# from skearln import model_C
from sklearn import neighbors 
from sklearn import linear_model

# TODO: Initialize the three models
clf_A = tree.DecisionTreeClassifier(max_depth=5)
clf_B = svm.SVC(kernel='rbf')
clf_C = neighbors.KNeighborsClassifier(n_neighbors=15)
clf_D = linear_model.LogisticRegression()

# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train.head(300)
y_train_300 = y_train.head(300)


# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

print "\nCalculating model A - Decision tree"
print "--------------------------------------"
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print "--------------------------------------\n"


print "\nCalculating model B - SVM (Support Vector Machines)"
print "--------------------------------------"
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print "--------------------------------------\n"


print "\nCalculating model C - KNN"
print "--------------------------------------"
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
print "--------------------------------------\n"



print "\nCalculating model D - logistic regression"
print "--------------------------------------"
train_predict(clf_D, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_D, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_D, X_train_300, y_train_300, X_test, y_test)
print "--------------------------------------\n"



Calculating model A - Decision tree
--------------------------------------
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0013 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.9861.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7132.


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0011 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.9184.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6917.


Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0013 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8716.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7194.
--------------------------------------


Calculating model B - SVM (Support Vector Machines)
--------------------------------------
Training a SVC using a training set size of 100. . .
Trained m

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0011                  | 0.0003                 | 0.9931           | 0.7087          |
| 200               | 0.0012                  | 0.0003                 | 0.9167           | 0.6866          |
| 300               | 0.0016                  | 0.0002                 | 0.8716           | 0.7101          |

** Classifer 2 - SVM (Support Vector Machines)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0015                  | 0.0008                 | 0.8471           | 0.7975          |
| 200               | 0.0039                  | 0.0015                 | 0.8319           | 0.7975          |
| 300               | 0.0079                  | 0.0022                 | 0.8080           | 0.7975          |

** Classifer 3 - KNN**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0007                  | 0.0017                 | 0.8402           | 0.7922          |
| 200               | 0.0008                  | 0.0021                 | 0.8359           | 0.7260          |
| 300               | 0.0008                  | 0.0026                 | 0.8033           | 0.7260          |


** Classifer 4 - Logistic regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0010                  | 0.0002                 | 0.9139           | 0.6290          |
| 200               | 0.0022                  | 0.0002                 | 0.8704           | 0.7244          |
| 300               | 0.0028                  | 0.0002                 | 0.7974           | 0.7518          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

The results from all of the models are not so good as I thought they will be. All of them are below 80%. I'm also quite surprised by the decreasing tendency of F1 score for all models for increasing training set. I would think that with larger training set, the F1 train score should be better.



From the performed tests it seems that the best results are from SVM and Logistic regression models. SVM takes relatively most of computational power during training and predicting, but the results are the best. On other side F1 test score from Logistic Regression is showing quite good results (increasing tendency with more training size) + the prediction time is short and constant.

Based on this I would choose Logistic regression

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

Chosen model is Logistic regression. 

Logistic regression makes classification predictions based on probability. As an input to the model are used all features. Predictions are made based on calculated linear or non-linear function (called hypothesis), which usually outputs values between 0 and 1. Usually all values that are equal or greater than 0.5 are considered as 1 output and all values below 0.5 is considered to have 0 output. This way the output of logisitic regression is 0 or 1. To achieve output values between 0 and 1 is usually used 'S' shaped function (for example sigmoid).


Every input feature will have assigned also a specific variable Theta, which is set/tuned during the training phase. For example if we have two features:  
- Age (x1)
- Ability to sing (x2)

and we want to predict happines of a person we will have following function (hypothetis) based on which we will calculate our final prediction:

h = Theta0 + (Theta1 * x1) + (Theta2 * x2)


Learning process is done by calculating a cost function which refers to difference between our predicted value and our real training value. 





### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [101]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer
from sklearn import linear_model
from sklearn import svm

# TODO: Create the parameters list you wish to tune
#parameters = {'kernel':('sigmoid', 'rbf'), 'C':[1, 10]}
parameters = {'solver':('liblinear', 'sag'), 'C':[1, 10]}

# TODO: Initialize the classifier
clf = linear_model.LogisticRegression()


# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')


# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)


# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)


# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0057 seconds.
Tuned model has a training F1 score of 0.8017.
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.7606.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

For Linear regression was final F1 training score for the tuned model 0.8017 and Testing F1 score 0.7606.
My untuned model gave me F1 score 0.7974 for training data set of 300, and F1 score for test 0.7518. 

We can see that we have got a bit better results from the tuned model (0.5%). 


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.